In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
import cv2
import os

In [2]:
from keras.models import Model
from keras.callbacks import *
from keras.layers import Input, Dense, Flatten,Activation, BatchNormalization, Conv2D, Conv2DTranspose, MaxPooling2D, Add, AveragePooling2D
from keras.activations import *
from keras.datasets import cifar,cifar10,cifar100
from keras.utils import to_categorical, plot_model
from keras.optimizers import *

In [3]:
# File Path
IMAGE_FOLDER_PATH = "train/train/"
FILE_PATH = [os.path.join(IMAGE_FOLDER_PATH,i) for i in os.listdir(IMAGE_FOLDER_PATH)]

# Labels
labels=[i for i in os.listdir(IMAGE_FOLDER_PATH)]

# Targets
targets=[i.split('.')[0] for i in labels]
y=targets

# Dataframe
data = pd.DataFrame({'PATH':FILE_PATH,'targets':targets}) 
data

,PATH,targets
0,train/train/cat.0.jpg,cat
1,train/train/cat.1.jpg,cat
2,train/train/cat.10.jpg,cat
3,train/train/cat.100.jpg,cat
4,train/train/cat.1000.jpg,cat
...,...,...
24995,train/train/dog.9995.jpg,dog
24996,train/train/dog.9996.jpg,dog
24997,train/train/dog.9997.jpg,dog
24998,train/train/dog.9998.jpg,dog


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=101)

In [6]:
val_df

,PATH,targets
7799,train/train/cat.5768.jpg,cat
4427,train/train/cat.2732.jpg,cat
14941,train/train/dog.12194.jpg,dog
11644,train/train/cat.9228.jpg,cat
15548,train/train/dog.1491.jpg,dog
...,...,...
20060,train/train/dog.5552.jpg,dog
12413,train/train/cat.9920.jpg,cat
10648,train/train/cat.8331.jpg,cat
598,train/train/cat.10535.jpg,cat


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
datagen = ImageDataGenerator()

In [9]:
train = datagen.flow_from_dataframe(dataframe=train_df, 
                                   x_col='PATH', 
                                   y_col='targets', 
                                   target_size=(222,222),
                                   class_mode='binary',
                                   rescale=1./255)

Found 20000 validated image filenames belonging to 2 classes.


In [10]:
train.class_indices

{'cat': 0, 'dog': 1}

In [11]:
val = datagen.flow_from_dataframe(dataframe=val_df, 
                                 x_col='PATH', 
                                 y_col='targets', 
                                 target_size=(222,222),
                                 class_mode='binary', 
                                 rescale=1./255)

Found 5000 validated image filenames belonging to 2 classes.


In [21]:
val.class_indices

{'cat': 0, 'dog': 1}

In [22]:
type(train)

keras.preprocessing.image.DataFrameIterator

# model creation

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
es = EarlyStopping(monitor='val_loss', patience=1, mode='min')

In [25]:
cl_in = Input(shape=(222,222,3))
cl_conv_1 = Conv2D(64, 3, strides=2, activation='selu', padding='same', kernel_initializer='lecun_normal')(cl_in)
cl_pl_1 = MaxPooling2D()(cl_conv_1)
cl_res_1_1 = BatchNormalization()(cl_pl_1)
cl_res_1_1 = Activation('relu')(cl_res_1_1)
cl_res_1_1 = Conv2D(64, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_1_1)
cl_res_1_2 = BatchNormalization()(cl_res_1_1)
cl_res_1_2 = Activation('relu')(cl_res_1_2)
cl_res_1_2 = Conv2D(64, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_1_2)
cl_res_1_a = Add()([cl_pl_1, cl_res_1_2])
cl_res_2_1 = BatchNormalization()(cl_res_1_a)
cl_res_2_1 = Activation('relu')(cl_res_2_1)
cl_res_2_1 = Conv2D(64, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_2_1)
cl_res_2_2 = BatchNormalization()(cl_res_2_1)
cl_res_2_2 = Activation('relu')(cl_res_2_2)
cl_res_2_2 = Conv2D(64, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_2_2)
cl_res_2_a = Add()([cl_res_1_a, cl_res_2_2])
cl_res_3_1 = BatchNormalization()(cl_res_2_a)
cl_res_3_1 = Activation('relu')(cl_res_3_1)
cl_res_3_1 = Conv2D(64, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_3_1)
cl_res_3_2 = BatchNormalization()(cl_res_3_1)
cl_res_3_2 = Activation('relu')(cl_res_3_2)
cl_res_3_2 = Conv2D(64, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_3_2)
cl_res_3_a = Add()([cl_res_2_a, cl_res_3_2])
cl_conv_2  = Conv2D(128, 3, strides=2, padding='same', kernel_initializer='lecun_normal')(cl_res_3_a)
cl_conv_2  = Conv2D(128, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_conv_2)
cl_res_4_1 = BatchNormalization()(cl_conv_2)
cl_res_4_1 = Activation('relu')(cl_res_4_1)
cl_res_4_1 = Conv2D(128, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_4_1)
cl_res_4_2 = BatchNormalization()(cl_res_4_1)
cl_res_4_2 = Activation('relu')(cl_res_4_2)
cl_res_4_2 = Conv2D(128, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_4_2)
cl_res_4_a = Add()([cl_conv_2, cl_res_4_2])
cl_res_5_1 = BatchNormalization()(cl_res_4_a)
cl_res_5_1 = Activation('relu')(cl_res_5_1)
cl_res_5_1 = Conv2D(128, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_5_1)
cl_res_5_2 = BatchNormalization()(cl_res_5_1)
cl_res_5_2 = Activation('relu')(cl_res_5_2)
cl_res_5_2 = Conv2D(128, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_5_2)
cl_res_5_a = Add()([cl_res_4_a, cl_res_5_2])
cl_res_6_1 = BatchNormalization()(cl_res_5_a)
cl_res_6_1 = Activation('relu')(cl_res_6_1)
cl_res_6_1 = Conv2D(128, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_6_1)
cl_res_6_2 = BatchNormalization()(cl_res_6_1)
cl_res_6_2 = Activation('relu')(cl_res_6_2)
cl_res_6_2 = Conv2D(128, 3, strides=1, padding='same', kernel_initializer='lecun_normal')(cl_res_6_2)
cl_res_6_a = Add()([cl_res_5_a, cl_res_6_2])
cl_pl_2 = AveragePooling2D()(cl_res_6_a)
cl_flatten = Flatten()(cl_pl_2)
cl_fc100 = Dense(100, activation='relu')(cl_flatten)
cl_fc10 = Dense(10, activation='relu')(cl_fc100)
cl_fc2 = Dense(1, activation='sigmoid')(cl_fc10)
model = Model(cl_in, cl_fc2)
plot_model(model, 'resnet_ReLUBN.png', show_layer_names=False)
model.summary()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 222, 222, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_30 (Conv2D)             (None, 111, 111, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 55

                                                                                                  
 batch_normalization_32 (BatchN  (None, 28, 28, 128)  512        ['add_15[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 activation_32 (Activation)     (None, 28, 28, 128)  0           ['batch_normalization_32[0][0]'] 
                                                                                                  
 conv2d_41 (Conv2D)             (None, 28, 28, 128)  147584      ['activation_32[0][0]']          
                                                                                                  
 batch_normalization_33 (BatchN  (None, 28, 28, 128)  512        ['conv2d_41[0][0]']              
 ormalization)                                                                                    
          

In [26]:
OPT = Adam(0.00001)
model.compile(loss='binary_crossentropy', optimizer=OPT, metrics=['binary_crossentropy', 'mse','acc'])
hist_classifier_ReLUBN_Res = model.fit(train, batch_size=64, epochs=15, validation_data=val, callbacks=[es])

Epoch 1/15
625/625 [==============================] - 717s 1s/step - loss: 0.7335 - binary_crossentropy: 0.7335 - mse: 0.2519 - acc: 0.5007 - val_loss: 0.6938 - val_binary_crossentropy: 0.6938 - val_mse: 0.2500 - val_acc: 0.4942
Epoch 2/15
625/625 [==============================] - 709s 1s/step - loss: 0.6932 - binary_crossentropy: 0.6932 - mse: 0.2499 - acc: 0.4989 - val_loss: 0.6943 - val_binary_crossentropy: 0.6943 - val_mse: 0.2502 - val_acc: 0.4932
